In [2]:
import import_ipynb
from phase1_node_to_code import node_to_code_app

/Users/kanishkgupta/Documents/GitHub/AgentAgent/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dict_for_node = {
    "planner": {
        "schema_info": """OverallState:
      type: TypedDict
      fields:
      - name: input
        type: str
      - name: plan
        type: List[str]
      - name: past_steps
        type: List[Tuple]
      - name: response
        type: str
      - name: messages
        type: Annotated[list[AnyMessage], add_messages]""",
    "input_schema": "OverallState",
    "output_schema":"OverallState",
    "description":"Plan step generates a plan based on the input using llm structured output functionality, stores it to the plan field",
    "function_name": "plan_step"
    },
    "math_node":{
        "schema_info": """MathState:
      type: TypedDict
      fields:
      - name: a
        type: int
      - name: b
        type: int
      - name: response
        type: int""",
    "input_schema": "MathState",
    "output_schema":"MathState",
    "description":"Gets the two numbers a and b from the state, and returns the sum of a and b., stores it to the response field",
    "function_name": "math_step"
    },
    "interrupt_node":{
        "schema_info": """RequirementAnalysisState:
      type: TypedDict
      fields:
      - name: user_input
        type: str
      - name: messages
        type: Annotated[list[AnyMessage], add_messages]""",
    "input_schema": "RequirementAnalysisState",
    "output_schema":"RequirementAnalysisState",
    "description":"Analyze the user input, see if they require any human intervention, if yes, return the human intervention required, else return no human intervention required",
    "function_name": "interrupt_step"
    },
    "decision_node":{
        "schema_info": """RequirementAnalysisState:
      type: TypedDict
      fields:
      - name: user_input
        type: str
      - name: messages
        type: Annotated[list[AnyMessage], add_messages]""",
    "input_schema": "RequirementAnalysisState",
    "output_schema":"RequirementAnalysisState",
    "description":"Analyze the user input which contain the requirements for building a product, acts as a decision node if more info is needed or not",
    "function_name": "decision_step"
    },
    "code_node":{
        "schema_info": """CodeWriterState:
      type: TypedDict
      fields:
      - name: user_query
        type: str
      - name: execution_result
        type: str""",
    "input_schema": "CodeWriterState",
    "output_schema":"RequiremenCodeWriterStatetAnalysisState",
    "description":"This node analyzes the user_query, if the query is to write a code, it will make a tool call to run the proposed code",
    "function_name": "code_step"
    },
    "weather_node":{
        "schema_info": """WeatherNodeState:
      type: TypedDict
      fields:
      - name: user_query
        type: str
      - name: execution_result
        type: str""",
    "input_schema": "WeatherNodeState",
    "output_schema":"WeatherNodeState",
    "description":"This node analyzes the user_query, makes tool calls to retrieve relevant information and answer the query",
    "function_name": "weather_step",
    },
    "stock_node":{
        "schema_info": """StockState:
      type: TypedDict
      fields:
      - name: user_query
        type: str
      - name: execution_result
        type: str""",
    "input_schema": "StockState",
    "output_schema":"StockState",
    "description":"This node analyzes the user_query related to finance/stock-markets, makes tool calls to retrieve relevant information and answer the query",
    "function_name": "stock_step",
    }
}


In [4]:
import uuid
uuid = uuid.uuid4()
config = {"configurable": {"thread_id": str(uuid)}}

for output in node_to_code_app.stream(dict_for_node["planner"], config, stream_mode="updates"
    ):
        print(output)

{'identify_node': {'node_type': 'planner', 'messages': [HumanMessage(content='\nYou are provided with the following information about the node:\n<SchemaInfo>\nOverallState:\n      type: TypedDict\n      fields:\n      - name: input\n        type: str\n      - name: plan\n        type: List[str]\n      - name: past_steps\n        type: List[Tuple]\n      - name: response\n        type: str\n      - name: messages\n        type: Annotated[list[AnyMessage], add_messages]\n</SchemaInfo>\n<InputSchema>\nOverallState\n</InputSchema>\n<OutputSchema>\nOverallState\n</OutputSchema>\n<Description>\nPlan step generates a plan based on the input using llm structured output functionality, stores it to the plan field\n</Description>\n<FunctionName>\nplan_step\n</FunctionName>\n\nBelow is the skeleton of the function that you need to implement:\ndef plan_step(state:OverallState) -> OverallState:\n    """Plan step generates a plan based on the input using llm structured output functionality, stores it

In [ ]:
from pprint import pprint
code= node_to_code_app.get_state(config).values["final_code"]
pprint(code)

In [ ]:
from phase1_node_to_code import replanner
steps= [("Generate a prompt that asks the LLM to evaluate the current progress based on the 'input' and 'past_steps'.",
     """
     Prompt generated:
      "Given the following input and past steps, evaluate the progress made so far and determine if the current plan needs revision.
      Please analyze the input: '[input]' and the past steps: '[past_steps]'.
      If the plan requires changes, outline a revised version.
      If the plan is sufficient, generate a final response based on the input and past steps."
     """)]
past_steps_summary = "\n".join([f"{index}. <step>{step}</step> <output>{output}</output>" for index, (step, output) in enumerate(steps)])

final_output= replanner.invoke({
    "plan":[ "Generate a prompt that asks the LLM to evaluate the current progress based on the 'input' and 'past_steps'.",
            "Use the LLM to either revise the existing plan or generate a final response based on the evaluation."],

"past_steps": past_steps_summary
    
     })

In [ ]:
from phase1_node_to_code import planner

planner.invoke({"messages":[(HumanMessage(content="""
You are provided with the following information about the node:
<SchemaInfo>
PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)
</SchemaInfo>
<InputSchema>
PlanExecute
</InputSchema>
<OutputSchema>
PlanExecute
</OutputSchema>
<Description>
Uses llm with tool binding for the stock related queries
</Description>
<FunctionName>
execute_step
</FunctionName>

Below is the skeleton of the function that you need to implement:
def execute_step(state:PlanExecute) -> PlanExecute:
    \"\"\"Uses llm with tool binding for the stock related queries\"\"\"
    # Implement the function to meet the description.

the state is of type PlanExecute and the function is of type PlanExecute
The general idea is that the implementation would involve extracting the input from the state, and updating the state with the output. Description contains the logic for this blackbox
                                          """))]})